<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> AC295: Advanced Practical Data Science </h1>

## Practicum 2: Visual Question Answering

**Harvard University, Fall 2020**  
**Instructors**: Pavlos Protopapas  

### **Team: $\alpha\beta normal$ $Distri\beta ution$**
#### **Roht Beri, Eduardo Peynetti, Jessica Wijaya, Stuart Neilson**

## Basic Model for Training

### Install Packages

In [1]:
!pip3 install transformers

     |████████████████████████████████| 1.3MB 13.4MB/s 
     |████████████████████████████████| 2.9MB 53.8MB/s 
     |████████████████████████████████| 1.1MB 52.0MB/s 
     |████████████████████████████████| 890kB 48.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7b57b32fa00c2cad22d3c04e8a079d02a556ea7d4bcceb8ef626c0842a5cc3dc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


### Imports

In [2]:
import os
import requests
import tempfile
import zipfile
import shutil
import json
import time
import sys
import cv2
import numpy as np
import pandas as pd
from collections import Counter
from glob import glob
from google.colab import drive
from tqdm.notebook import trange, tqdm
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.utils.layer_utils import count_params
from tensorflow_addons.metrics import F1Score

from transformers import BertTokenizer, TFBertModel

### Download Data

In [53]:
"""
!mkdir 'data'
#shutil.rmtree('/content/data/train2014_tf')
#shutil.rmtree('/content/data/val2014_tf')
!cp -r '/content/drive/My Drive/Practicum2Data/big/val2014_tf' /content/data
!cp -r '/content/drive/My Drive/Practicum2Data/big/train2014_tf' /content/data
!cp -r "/content/drive/My Drive/Practicum2Data/big/answers.csv" /content/data
"""
if not os.path.exists('/content/data'):
    os.mkdir('/content/data')

!gsutil cp -r gs://practicum2-abnormal-distribution/train2014_tf /content/data
!gsutil cp -r gs://practicum2-abnormal-distribution/val2014_tf /content/data

# https://storage.googleapis.com/practicum2-abnormal-distribution/train2014_tf
# https://storage.googleapis.com/practicum2-abnormal-distribution/val2014_tf

Copying gs://practicum2-abnormal-distribution/train2014_tf/vaq_raw_train2014_00-of-10.records...
Copying gs://practicum2-abnormal-distribution/train2014_tf/vaq_raw_train2014_01-of-10.records...
Copying gs://practicum2-abnormal-distribution/train2014_tf/vaq_raw_train2014_02-of-10.records...
Copying gs://practicum2-abnormal-distribution/train2014_tf/vaq_raw_train2014_03-of-10.records...
| [4 files][  2.4 GiB/  2.4 GiB]   72.2 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://practicum2-abnormal-distribution/train2014_tf/vaq_raw_train2014_04-of-10.records...
Copying gs://practicum2-abnormal-distribution/train2014_tf/vaq_raw_train2014_05-of-10.records...
Copying gs://practicum2-abnormal-distribution/train2014_tf/vaq_raw_train2014_06-of-10

In [3]:
drive.mount('/content/drive', force_remount=False)
if not os.path.exists('/content/data'):
    os.mkdir('/content/data')

!cp -r '/content/drive/My Drive/Practicum2Data/big' /content/data

Mounted at /content/drive


### Utils

In [4]:
# we use the following to save the models
class JsonEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, decimal.Decimal):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return super(JsonEncoder, self).default(obj)

# save_model saves everything. weights, statuses and results. 
def save_model(model,training_results,execution_time, learning_rate, epochs, optimizer, evaluation_results,path="models"):
  model_name=model.name
  # Get the model train history
  model_train_history = training_results.history

  # Ensure path exists
  if not os.path.exists(path):
      os.mkdir(path)

  # Save the enitire model (structure + weights)
  model.save(os.path.join(path,model_name+".hdf5"))

  # Save only the weights
  model.save_weights(os.path.join(path,model_name+".h5"))

  # Save the structure only
  model_json = model.to_json()
  with open(os.path.join(path,model_name+".json"), "w") as json_file:
      json_file.write(model_json)
    
  model_size = get_model_size(model_name=model_name)

  # Save model history
  with open(os.path.join("models",model_name+"_train_history.json"), "w") as json_file:
      json_file.write(json.dumps(model_train_history,cls=JsonEncoder))

  trainable_parameters = count_params(model.trainable_weights)
  non_trainable_parameters = count_params(model.non_trainable_weights)
  total_params = trainable_parameters + non_trainable_parameters

  # Save model metrics
  metrics ={
      "total_params":total_params,
      "execution_time":execution_time,
      "loss":evaluation_results[0],
      "accuracy":evaluation_results[1],
      "model_size":model_size,
      "learning_rate":learning_rate,
      "epochs":epochs,
      "optimizer":type(optimizer).__name__,
      "name": model_name,
      "id": int(time.time())
  }

  with open(os.path.join("models",model.name+"_metrics.json"), "w") as json_file:
      json_file.write(json.dumps(metrics,cls=JsonEncoder))

def get_model_size(path="models",model_name="model01"):
  model_size = os.stat(os.path.join(path,model_name+".hdf5")).st_size
  return model_size

def evaluate_model(model,test_data, training_results,execution_time, learning_rate, epochs, 
                   optimizer,save=True, 
                   loss_metrics=["loss","val_loss"],
                   acc_metrics=["accuracy","val_accuracy"]):
    
  # Get the model train history
  model_train_history = training_results.history
  # Get the number of epochs the training was run for
  num_epochs = len(model_train_history[loss_metrics[0]])

  # Plot training results
  fig = plt.figure(figsize=(15,5))
  axs = fig.add_subplot(1,2,1)
  axs.set_title('Loss')
  # Plot all metrics
  for metric in loss_metrics:
      axs.plot(np.arange(0, num_epochs), model_train_history[metric], label=metric)
  axs.legend()
  
  axs = fig.add_subplot(1,2,2)
  axs.set_title('Accuracy')
  # Plot all metrics
  for metric in acc_metrics:
      axs.plot(np.arange(0, num_epochs), model_train_history[metric], label=metric)
  axs.legend()

  plt.show()
  
  # Evaluate on test data
  evaluation_results = model.evaluate(test_data, return_dict=True)
  print(evaluation_results)

  evaluation_results = [evaluation_results[loss_metrics[0]], evaluation_results[acc_metrics[0]]]
  
  if save:
      # Save model
      save_model(model,training_results,execution_time, learning_rate, epochs, optimizer, evaluation_results)
  
  return evaluation_results

In [5]:
# Constants
IMG_WIDTH = 224
IMG_HEIGHT = 224
IMG_CHANNELS = 3

K = 10

AUTOTUNE = tf.data.experimental.AUTOTUNE

# Pipeline variables
batch_size = 8
train_buffer_size = 1000
val_buffer_size = 200
prefetch = AUTOTUNE

In [6]:
# Get Top K answers
def get_top_K_answers(k):
    answers = pd.read_csv("/content/data/big/answers.csv", index_col=0)
    answers = answers.index[:k]
    return list(answers)

In [7]:
# Function to parse data features
def _parse_features_function(example):
    # Parse the input tf.train.Example proto using the dictionary above.
    tf_records_features = {
        'image_raw': tf.io.FixedLenFeature([], tf.string), 
        'question' : tf.io.FixedLenFeature([], tf.string),
        'input_ids': tf.io.FixedLenFeature([], tf.string),
        'token_type_ids': tf.io.FixedLenFeature([], tf.string),
        'attention_mask': tf.io.FixedLenFeature([], tf.string), 
        'answer': tf.io.FixedLenFeature([], tf.string)
    }
    return tf.io.parse_single_example(example, tf_records_features)


# Filter if answer is no
def filter_fn(x):
    #use broadcasting for element-wise tensor operation
    broadcast_equal = tf.equal(TOP_ANSWERS, x['answer'])
    broadcast_equal_int = tf.cast(broadcast_equal, tf.int8)
    broadcast_sum = tf.reduce_sum(broadcast_equal_int)
    return broadcast_sum > 0


# Read image and resize it
def read_and_decode(img):
    img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
    img = tf.cast(img, tf.float32)/255.0
    return img


# Structure the data for training
def structure_data(data):
    image = data['image_raw']
    image = read_and_decode(image)
    
    input_ids = tf.io.decode_raw(data['input_ids'], tf.int32)
    attention_mask = tf.io.decode_raw(data['attention_mask'], tf.int32)
    token_type_ids = tf.io.decode_raw(data['token_type_ids'], tf.int32)
    
    answer = tf.io.decode_raw(data['answer'], tf.int32)

    return ((image, (input_ids, token_type_ids, attention_mask)), answer)

### Build Pipeline

In [8]:
# ############## #
# # Train data # #
# ############## #
tfrecords_pattern_path = "/content/data/big/train2014_tf/vaq_raw_train2014_*-of-*.records"
train_files = tf.io.matching_files(tfrecords_pattern_path)
train_files = tf.random.shuffle(train_files)
train_shards = tf.data.Dataset.from_tensor_slices(train_files)

train = train_shards.interleave(tf.data.TFRecordDataset)
train = train.map(_parse_features_function, num_parallel_calls=AUTOTUNE)
#train = train.filter(filter_fn)
train = train.map(structure_data, num_parallel_calls=AUTOTUNE)
train = train.shuffle(buffer_size=train_buffer_size).batch(batch_size)
train = train.cache().prefetch(prefetch)

# ################### #
# # Validation data # #
# ################### #
tfrecords_pattern_path = "/content/data/big/val2014_tf/vaq_raw_val2014_*-of-*.records"
val_files = tf.io.matching_files(tfrecords_pattern_path)
val_files = tf.random.shuffle(val_files)
val_shards = tf.data.Dataset.from_tensor_slices(val_files)

valid = val_shards.interleave(tf.data.TFRecordDataset)

valid = valid.map(_parse_features_function, num_parallel_calls=AUTOTUNE)
#valid = valid.filter(filter_fn)
valid = valid.map(structure_data, num_parallel_calls=AUTOTUNE)
valid = valid.shuffle(buffer_size=val_buffer_size).batch(batch_size)
valid = valid.cache().prefetch(prefetch)

### Get Top Answers

In [9]:
top_answers = get_top_K_answers(K)
#top_answers = tf.constant(top_answers)

### Build Model

In [10]:
def build_vqa_model(image_height, image_width, num_channels, num_classes):
    # Handle to pretrained model (Use a different model here)
    input_shape=[image_height, image_width, num_channels]
    resnet = keras.applications.InceptionResNetV2(
        include_top=False, 
        weights='imagenet', 
        input_shape=input_shape
    )
    resnet.trainable = False
    image_hidden_states = layers.Flatten()(resnet.output)

    input_ids = layers.Input(shape=(24,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(24,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(24,), dtype=tf.int32)
    bert = TFBertModel.from_pretrained('bert-base-uncased')
    bert.trainable = False
    question = bert(
        input_ids, 
        token_type_ids=token_type_ids, 
        attention_mask=attention_mask
    )
    last_hidden_states = layers.Flatten()(question[0])

    x = layers.concatenate([image_hidden_states, last_hidden_states])
    
    # Regularize using L1
    kernel_weight = 0.02
    bias_weight = 0.02

    """
    x = layers.Dense(
        units=64, 
        activation='relu',
        kernel_regularizer=regularizers.l1(kernel_weight), 
        bias_regularizer=regularizers.l1(bias_weight))(x)
    """

    output = layers.Dense(
        units=num_classes,
        kernel_regularizer=tf.keras.regularizers.l1(kernel_weight), 
        bias_regularizer=tf.keras.regularizers.l1(bias_weight))(x)

    model = Model(inputs=[resnet.input, (input_ids, token_type_ids, attention_mask)], outputs=output)

    # Optimizer
    optimizer = optimizers.Adam(lr=0.001)

    # Loss
    loss = losses.CategoricalCrossentropy(from_logits=True)

    # Compile
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=['accuracy']
    )

    return model

In [11]:
model = build_vqa_model(224, 224, 3, K)

219062272/219055592 [==============================] - 1s 0us/step


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# Train model
start_time = time.time()

training_results = model.fit(
        train,
        validation_data=valid,
        epochs=10, 
        verbose=1)

execution_time = (time.time() - start_time)/60.0
print("Training execution time (mins)",execution_time)

evaluate_save_model(
    model, 
    valid, 
    training_results, 
    execution_time, 0.001, 
    batch_size, 
    10, 
    optimizers.Adam(lr=0.001),
    save=True
)

Epoch 1/10
   1715/Unknown - 146s 85ms/step - loss: 2558149525504.0000 - accuracy: 0.3797